сайт соревнования Semeval-2021 Task 2: [Multilingual and Cross-lingual Word-in-Context Disambiguation](https://competitions.codalab.org/competitions/27054)

In [1]:
!git clone -q  https://github.com/DanilDmitriev1999/ABBYY_HW_NLP

In [2]:
!pip install --upgrade -r /content/ABBYY_HW_NLP/WiC/requirements.txt

     |████████████████████████████████| 7.0MB 9.2MB/s 
     |████████████████████████████████| 12.8MB 209kB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 696kB 32.0MB/s 
     |████████████████████████████████| 2.3MB 32.2MB/s 
     |████████████████████████████████| 1.2MB 29.7MB/s 
     |████████████████████████████████| 22.3MB 9.0MB/s 
     |████████████████████████████████| 266kB 35.6MB/s 
     |████████████████████████████████| 276kB 24.2MB/s 
     |████████████████████████████████| 829kB 31.5MB/s 
     |████████████████████████████████| 112kB 39.5MB/s 
     |████████████████████████████████| 901kB 28.7MB/s 
     |████████████████████████████████| 3.3MB 29.8MB/s 
     |████████████████████████████████| 522kB 39.6MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 1.3MB 27.0MB/s 
     |████████████████████████████████| 296kB 40.3MB/

In [3]:
import warnings
warnings.filterwarnings('ignore')
import comet_ml

import os
import torch
import random
import numpy as np
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
from torch.utils.data import Dataset, RandomSampler, DataLoader
from torch import nn
from torch.nn import functional as F

from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.metrics import Accuracy
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import EarlyStopping  
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup, AutoModel
from transformers import BertTokenizerFast


comet_ml is installed but `COMET_API_KEY` is not set.


In [4]:
from ABBYY_HW_NLP.WiC.DataModule.custom_dataset import *
from ABBYY_HW_NLP.WiC.DataModule.utils import *

In [5]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

### Скачиваем данные с гитхаба соревнования

In [6]:
!mkdir data && git clone https://github.com/SapienzaNLP/mcl-wic.git data && cd data && unzip 'SemEval-2021_MCL-WiC_all-datasets.zip'  && mv 'SemEval-2021_MCL-WiC_test-gold-data.zip' 'MCL-WiC/test' && cd MCL-WiC/test && unzip 'SemEval-2021_MCL-WiC_test-gold-data.zip' && rm *.zip && cd ../.. && rm *.zip && ls


Cloning into 'data'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 188 (delta 41), reused 0 (delta 0), pack-reused 96
Receiving objects: 100% (188/188), 7.59 MiB | 15.89 MiB/s, done.
Resolving deltas: 100% (68/68), done.
Archive:  SemEval-2021_MCL-WiC_all-datasets.zip
   creating: MCL-WiC/
   creating: MCL-WiC/training/
  inflating: MCL-WiC/training/training.en-en.data  
  inflating: MCL-WiC/training/training.en-en.gold  
  inflating: MCL-WiC/README.txt      
   creating: MCL-WiC/test/
   creating: MCL-WiC/test/crosslingual/
  inflating: MCL-WiC/test/crosslingual/test.en-ar.data  
  inflating: MCL-WiC/test/crosslingual/test.en-zh.data  
  inflating: MCL-WiC/test/crosslingual/test.en-ru.data  
  inflating: MCL-WiC/test/crosslingual/test.en-fr.data  
   creating: MCL-WiC/test/multilingual/
  inflating: MCL-WiC/test/multilingual/test.fr-fr.data  
  inflating: MCL-WiC/test/multilingual/

In [7]:
df_train_wic = read_train_data('data/MCL-WiC/training/training.en-en.data', True)
df_dev_wic = read_train_data('data/MCL-WiC/dev/multilingual/dev.en-en.data', True)
df_test_wic = read_train_data('data/MCL-WiC/test/multilingual/test.en-en.data', True)
print(df_train_wic.shape)
print(df_dev_wic.shape)
print(df_test_wic.shape)
print(df_train_wic.head())
print(df_dev_wic.head())
print(df_test_wic.head())

(8000, 10)
(1000, 10)
(1000, 10)
                 id   lemma   pos  ... start2 end2 tag
0  training.en-en.0    play  NOUN  ...     10   14   0
1  training.en-en.1    play  NOUN  ...    112  116   0
2  training.en-en.2  esteem  NOUN  ...    106  112   1
3  training.en-en.3  esteem  NOUN  ...     20   26   1
4  training.en-en.4  holder  NOUN  ...     27   33   1

[5 rows x 10 columns]
            id         lemma   pos  ... start2 end2 tag
0  dev.en-en.0      superior  NOUN  ...     41   50   0
1  dev.en-en.1      superior  NOUN  ...     44   53   1
2  dev.en-en.2  acquaintance  NOUN  ...     41   54   0
3  dev.en-en.3  acquaintance  NOUN  ...     74   86   0
4  dev.en-en.4       baggage  NOUN  ...      6   13   1

[5 rows x 10 columns]
             id   lemma   pos  ... start2 end2 tag
0  test.en-en.0  gently   ADV  ...     59   65   0
1  test.en-en.1  gently   ADV  ...     72   78   0
2  test.en-en.2    next   ADJ  ...     85   89   1
3  test.en-en.3    next   ADJ  ...     84   88   1


### Разбиваем по леммам

In [8]:
global_df = pd.concat([df_train_wic, df_dev_wic], ignore_index=True)
df_train, df_val = lemma_train_test_split(global_df)

df_test = df_test_wic
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)


(7176, 10)
(1824, 10)
(1000, 10)


In [9]:
train_labels = df_train["tag"].tolist()
val_labels = df_val["tag"].tolist()
test_labels = df_test["tag"].tolist()

train_sentences = [(s1, s2) for s1, s2 in zip(df_train["sentence1"], df_train["sentence2"])]
val_sentences = [(s1, s2) for s1, s2 in zip(df_val["sentence1"], df_val["sentence2"])]
test_sentences = [(s1, s2) for s1, s2 in zip(df_test["sentence1"], df_test["sentence2"])]

train_word_ranges = [((int(s1), int(e1)), (int(s2), int(e2))) for s1, e1, s2, e2 in zip(df_train["start1"], df_train["end1"], df_train["start2"], df_train["end2"])]
val_word_ranges = [((int(s1), int(e1)), (int(s2), int(e2))) for s1, e1, s2, e2 in zip(df_val["start1"], df_val["end1"], df_val["start2"], df_val["end2"])]
test_word_ranges = [((int(s1), int(e1)), (int(s2), int(e2))) for s1, e1, s2, e2 in zip(df_test["start1"], df_test["end1"], df_test["start2"], df_test["end2"])]

print(train_sentences[0])
print(train_word_ranges[0])

('We would also like to convey our esteem and congratulations to fraternal Lebanon and its people on the unconditional liberation of its southern part.', 'Father Lini said that, because of that, the United Nations has a very special place in the affections and esteem of the people of Vanuatu.')
((33, 39), (106, 112))


### Готовим Dataset и Dataloader

In [10]:
INDICES_PADDING_LEN = 5
INDICES_PADDING_VALUE = 0
MAX_TOKENS = 100
BATCH_SIZE = 8
EPOCHS = 5

In [11]:
train_data = BertDataset(train_sentences, train_word_ranges, train_labels, MAX_TOKENS)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler)

val_data = BertDataset(val_sentences, val_word_ranges, val_labels, MAX_TOKENS)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

test_data = BertDataset(test_sentences, test_word_ranges, test_labels, MAX_TOKENS)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)


In [12]:
len(test_loader)

125

In [13]:
def batched_index_select(t, dim, inds):
    dummy = inds.unsqueeze(2).expand(inds.size(0), inds.size(1), t.size(2))
    out = t.gather(dim, dummy) # b x e x f
    return out

def get_mask(indices, embedding_size):
    mask = (indices != INDICES_PADDING_VALUE)
    mask.unsqueeze_(-1)
    mask = mask.expand(mask.shape[0], mask.shape[1], embedding_size)
    LARGE_VALUE = 2**32
    return torch.where(mask == True, 0, LARGE_VALUE)

def get_tokens_embeddings(batch, indices):
    return batched_index_select(batch, 1, indices) - get_mask(indices, batch.shape[2])


### используйте косинусную близость вместо линейных слоев для определения, имеют ли таргет-слова одно значение или разные

In [18]:
comet_logger = CometLogger(
    api_key="HWfJT3eyByVJWe4nEbi1pGosA",
    workspace="danildmitriev1999",
    project_name="3-sem",
    experiment_name="BERT with CosSim, RELU ",
)

CometLogger will be initialized in online mode


In [19]:
class CosSimClassifier(LightningModule):
    def __init__(self, model_path):
        super(CosSimClassifier, self).__init__()

        self.model = AutoModel.from_pretrained(model_path)
        self.embedding_dim = self.model.get_input_embeddings().embedding_dim

        self.loss = nn.BCELoss()
        self.save_hyperparameters()

        self.valid_accuracy = Accuracy()
        self.test_accuracy = Accuracy()
        self.predictions_proba = torch.Tensor()

        # self.input_dim = 2 * self.embedding_dim
        # self.linear = nn.Linear(self.input_dim, 10)
        # self.final_linear = nn.Linear(10, 1)

        self.relu = nn.ReLU()
        # self.relu = nn.ReLU()

        self.cos_sim = nn.CosineSimilarity()

        self.predictions_proba = []

        self.threshold = 0.678

    def _get_embeddings(self, input_ids, attention_mask, word_indices):
        sentence_outputs = self.model(input_ids, attention_mask).last_hidden_state
        tokens_embeddings = get_tokens_embeddings(sentence_outputs, word_indices)
        word_embedding = torch.max(tokens_embeddings, 1)[0]
        return word_embedding

    def forward(self, input_ids, attention_mask, word_indices):
        first_embeddings = self._get_embeddings(input_ids[0], attention_mask[0], word_indices[0])
        second_embeddings = self._get_embeddings(input_ids[1], attention_mask[1], word_indices[1])

        outputs = self.cos_sim(first_embeddings, second_embeddings)
        outputs = self.relu(outputs)
        return outputs    

    def training_step(self, batch, _):
        inputs, attn, word_indices, labels = batch
        outputs = self(inputs, attn, word_indices)
        return self.loss(outputs, labels)

    def validation_step(self, batch, _):
        inputs, attn, word_indices, labels = batch
        outputs = self(inputs, attn, word_indices)

        logits = (outputs > self.threshold).float()

        self.valid_accuracy.update(logits, labels.int())
        self.log("val_acc", self.valid_accuracy)

        loss = self.loss(outputs, labels)
        self.log("val_loss", loss, prog_bar=True)

    def validation_epoch_end(self, _):
        self.log("val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, attn, word_indices, labels = batch
        outputs = self(inputs, attn, word_indices)
        self.predictions_proba.append(outputs)

        logits = (outputs > self.threshold).float()

        self.test_accuracy.update(logits, labels.int())
        self.log("test_acc", self.test_accuracy)

        loss = self.loss(outputs, labels)
        self.log("test_loss", loss)
    
    def test_epoch_end(self, _):
        self.log("test_acc_epoch", self.test_accuracy.compute())

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=1e-5)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=TOTAL_STEPS)
        return [optimizer], [scheduler]

In [20]:
TOTAL_STEPS = (len(train_data) // BATCH_SIZE + 1) * EPOCHS

# bert_model = BertModel.from_pretrained("bert-base-cased").to('cuda')
clf = CosSimClassifier("bert-base-cased")

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=2,
    verbose=True,
    mode="min" 
)

trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=10,
    max_epochs=EPOCHS,
    callbacks=[early_stop_callback],
    logger=[comet_logger],
    val_check_interval=0.5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [21]:
trainer.fit(clf, train_loader, val_loader)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/danildmitriev1999/3-sem/17e7ded3b32349a29b7ec7ccfb82609b


  | Name           | Type             | Params
----------------------------------------------------
0 | model          | BertModel        | 108 M 
1 | loss           | BCELoss          | 0     
2 | valid_accuracy | Accuracy         | 0     
3 | test_accuracy  | Accuracy         | 0     
4 | relu           | ReLU             | 0     
5 | cos_sim        | CosineSimilarity | 0     
----------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/danildmitriev1999/3-sem/17e7ded3b32349a29b7ec7ccfb82609b
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [270]        : (0.01891769841313362, 0.10959845036268234)
COMET INFO:     val_acc [6]       : (0.5767543911933899, 0.6842105388641357)
COMET INFO:     val_acc_epoch [6] : (0.5767543911933899, 0.6842105388641357)
COMET INFO:     val_loss [6]      : (0.4938918650150299, 0.602857232093811)
COMET INFO:   Others:
COMET INFO:     Name : BERT with CosSim, RELU 
COMET INFO:   Parameters:
COMET INFO:     model_path : bert-base-cased
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     model graph         : 1
COMET INFO:     notebook  

COMET INFO: Uploading 1 metrics, params and output messages


1

In [22]:
trainer.test(clf, test_loader)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/danildmitriev1999/3-sem/17e7ded3b32349a29b7ec7ccfb82609b




--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.6880000233650208,
 'test_acc_epoch': 0.6880000233650208,
 'test_loss': 0.5656610131263733}
--------------------------------------------------------------------------------


[{'test_acc': 0.6880000233650208,
  'test_acc_epoch': 0.6880000233650208,
  'test_loss': 0.5656610131263733}]